In [2]:

import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score
import warnings
warnings.filterwarnings('ignore')
# Preparing Data
import pandas as pd
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :60]
x_data = x_data.drop('id', axis=1)
y_data = df['target']
##### Get Base Scoring
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import*
kfold = StratifiedKFold(n_splits=10, random_state=None)
model = RandomForestClassifier()
scoring = 'neg_log_loss'
results = cross_val_score(model, x_data, y_data, cv=kfold, scoring=scoring)
print(results.mean())
def objective(trial):
    # Define hyperparameters to optimize
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(59,), (59, 59,), (59,59,59),(30,), (30, 30,), (30,30,30),  ])
    activation = trial.suggest_categorical('activation', ['identity', 'logistic', 'tanh', 'relu'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam'])
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-2)
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive'])
    # Create the MLPClassifier
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate=learning_rate,
        max_iter=100
    )
    # Evaluate using cross-validation
    score = cross_val_score(mlp, x_data, y_data, cv=kfold, scoring=scoring).mean()
    return score
# Optimize hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, n_jobs=-1)
# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)
print("Best Score", study.best_value)


mlp = MLPClassifier(**best_params, max_iter=5, verbose=False)
results = cross_val_score(mlp, x_data, y_data, cv=kfold, scoring=scoring)
print(results.mean())



[I 2024-10-08 19:00:21,824] A new study created in memory with name: no-name-0a8fc8c9-4a8b-4b03-a1fd-73c5d3640a4b


-0.5342909504897584


[I 2024-10-08 19:00:24,987] Trial 2 finished with value: -0.7029204590976001 and parameters: {'hidden_layer_sizes': (30, 30, 30), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0006551789406043006, 'learning_rate': 'invscaling'}. Best is trial 2 with value: -0.7029204590976001.
[I 2024-10-08 19:00:26,471] Trial 11 finished with value: -0.7108643931055807 and parameters: {'hidden_layer_sizes': (59, 59, 59), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.00020942647918790214, 'learning_rate': 'invscaling'}. Best is trial 2 with value: -0.7029204590976001.
[I 2024-10-08 19:00:29,105] Trial 1 finished with value: -0.710460953183539 and parameters: {'hidden_layer_sizes': (30, 30, 30), 'activation': 'identity', 'solver': 'sgd', 'alpha': 0.0007816071637567015, 'learning_rate': 'invscaling'}. Best is trial 2 with value: -0.7029204590976001.
[I 2024-10-08 19:00:29,352] Trial 4 finished with value: -4.058106619688504 and parameters: {'hidden_layer_sizes': (30,), 'activation': 'logistic', '

Best hyperparameters: {'hidden_layer_sizes': (59,), 'activation': 'identity', 'solver': 'adam', 'alpha': 0.00019659725644940586, 'learning_rate': 'constant'}
Best Score -0.5659031696950227
-0.7130973370041526
